In [6]:
using TuringModels

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
size(d) # Should be 504x8

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/r03H1/src/Turing.jl:24


(504, 8)

pulled_left, actors, condition, prosoc_left

In [7]:
@model m10_4(y, actors, x₁, x₂) = begin
    # Number of unique actors in the data set
    N_actor = length(unique(actors))
    # Set an TArray for the priors/param
    α = TArray{Any}(undef, N_actor)
    # For each actor [1,..,7] set a prior
    for i ∈ 1:length(α)
        α[i] ~ Normal(0,10)
    end

    βp ~ Normal(0, 10)
    βpC ~ Normal(0, 10)

    for i ∈ 1:length(y)
        p = logistic(α[actors[i]] + (βp + βpC * x₁[i]) * x₂[i])
        y[i] ~ Binomial(1, p)
    end
end;

posterior = sample(m10_4(d[:,:pulled_left], d[:,:actor],d[:,:condition],
d[:,:prosoc_left]), Turing.NUTS(2000, 1000, 0.95));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/r03H1/src/inference/support/hmc_core.jl:240
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.4
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/r03H1/src/inference/support/hmc_core.jl:235
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/r03H1/src/core/ad.jl:169
┌ Warning: grad = [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/r03H1/src/core/ad.jl:170
┌ Info:  Adapted ϵ = 0.09340741377838617, std = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/r03H1/src/inference/adapt/adapt.jl:91
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/r03H1/src/core/ad.jl:169
┌ Warning: grad = [-1.413683823692169, NaN, -0.0053143216643326

[NUTS] Finished with
  Running time        = 1036.9406298429983;
  #lf / sample        = 0.0;
  #evals / sample     = 67.339;
  pre-cond. metric    = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,....


Fix the inclusion of adaptation samples

In [8]:
draws = 1001:2000
posterior2 = Chains(posterior[draws,:,:], :parameters)

Object of type Chains, with data of type 1000×9×1 Array{Union{Missing, Float64},3}

Log evidence      = 0.0
Iterations        = 1001:2000
Thinning interval = 1
Chains            = Chain1
Samples per chain = 1000
parameters        = α[2], βpC, βp, α[7], α[4], α[6], α[5], α[3], α[1]

parameters
       Mean    SD   Naive SE  MCSE     ESS   
α[1] -0.7368 0.2626   0.0083 0.0096  743.4117
α[2] 11.1931 5.4292   0.1717 0.8311   42.6710
α[3] -1.0455 0.2848   0.0090 0.0076 1000.0000
α[4] -1.0465 0.2808   0.0089 0.0065 1000.0000
α[5] -0.7435 0.2715   0.0086 0.0084 1000.0000
α[6]  0.2216 0.2738   0.0087 0.0065 1000.0000
α[7]  1.8233 0.3832   0.0121 0.0146  692.2298
  βp  0.8450 0.2572   0.0081 0.0115  500.4234
 βpC -0.1528 0.2815   0.0089 0.0101  782.8304



Rethinking/CmdStan results

In [9]:
m_10_04s_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
        Mean        SD       Naive SE       MCSE      ESS
a.1 -0.74503184 0.26613979 0.0042080396 0.0060183398 1000
a.2 10.77955494 5.32538998 0.0842018089 0.1269148045 1000
a.3 -1.04982353 0.28535997 0.0045119373 0.0049074219 1000
a.4 -1.04898135 0.28129307 0.0044476339 0.0056325117 1000
a.5 -0.74390933 0.26949936 0.0042611590 0.0052178124 1000
a.6  0.21599365 0.26307574 0.0041595927 0.0045153523 1000
a.7  1.81090866 0.39318577 0.0062168129 0.0071483527 1000
bp  0.83979926 0.26284676 0.0041559722 0.0059795826 1000
bpC -0.12913322 0.29935741 0.0047332562 0.0049519863 1000
";

Describe the draws

In [10]:
describe(posterior2)

Log evidence      = 0.0
Iterations        = 1001:2000
Thinning interval = 1
Chains            = Chain1
Samples per chain = 1000
parameters        = α[2], βpC, βp, α[7], α[4], α[6], α[5], α[3], α[1]



┌ Warning: `quantile(v::AbstractArray{<:Real})` is deprecated, use `quantile(v, [0.0, 0.25, 0.5, 0.75, 1.0])` instead.
│   caller = (::getfield(MCMCChains, Symbol("##102#104")){Chains{Union{Missing, Float64},Float64,NamedTuple{(:parameters,),Tuple{Array{String,1}}},NamedTuple{(:hashedsummary,),Tuple{Base.RefValue{Tuple{UInt64,MCMCChains.ChainSummaries}}}}}})(::String) at none:0
└ @ MCMCChains ./none:0


Empirical Posterior Estimates
─────────────────────────────────────────────
parameters
       Mean    SD   Naive SE  MCSE     ESS   
α[1] -0.7368 0.2626   0.0083 0.0096  743.4117
α[2] 11.1931 5.4292   0.1717 0.8311   42.6710
α[3] -1.0455 0.2848   0.0090 0.0076 1000.0000
α[4] -1.0465 0.2808   0.0089 0.0065 1000.0000
α[5] -0.7435 0.2715   0.0086 0.0084 1000.0000
α[6]  0.2216 0.2738   0.0087 0.0065 1000.0000
α[7]  1.8233 0.3832   0.0121 0.0146  692.2298
  βp  0.8450 0.2572   0.0081 0.0115  500.4234
 βpC -0.1528 0.2815   0.0089 0.0101  782.8304

Quantiles
─────────────────────────────────────────────
parameters
       2.5%   25.0%   50.0%   75.0%   97.5% 
α[1] -1.7078 -0.9135 -0.7309 -0.5709  0.1285
α[2]  3.0412  7.2461 10.0285 14.0184 31.8224
α[3] -2.1025 -1.2400 -1.0375 -0.8606 -0.1937
α[4] -2.0072 -1.2185 -1.0397 -0.8696 -0.0853
α[5] -1.5113 -0.9400 -0.7376 -0.5517  0.0889
α[6] -0.6183  0.0334  0.2297  0.4092  1.1575
α[7]  0.7556  1.5585  1.8237  2.0891  3.1643
  βp  0.1388  0.6579  0.8

Create a DataFrame

In [11]:
cnames = [
  :a_1, :a_2, :a_3, :a_4, :a_5, :a_6, :a_7,
  :bp, :bpC
]

df = DataFrame(convert(Matrix{Float64}, to_df(posterior2)), cnames)
first(df, 5)

,a_1,a_2,a_3,a_4,a_5,a_6,a_7,bp,bpC
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-1.0929,6.73563,-0.976573,-1.12526,-0.886459,0.322946,2.6305,0.850382,-0.395548
2,-0.431177,6.35147,-1.23014,-1.1136,-0.704243,-0.0204245,1.48751,0.558974,0.0371371
3,-0.732745,4.9389,-0.913287,-1.05535,-0.778746,0.569156,2.39251,0.942476,-0.376117
4,-0.647605,4.52435,-0.908345,-0.982594,-0.555483,-0.234643,1.56546,1.02727,-0.187141
5,-0.518945,8.02371,-1.03367,-0.914572,-0.562902,-0.145583,2.15204,0.586953,-0.084613


End of 10/m_10_04t.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*